In [37]:
from paper_wallet.wallet import PaperWallet
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import requests
import time
from keras.models import load_model

In [38]:
model = load_model('models/model.keras')
#Read test data
df = pd.read_csv('model performance stats/prices.log', header=None, names=["Price", "Timestamp"])
prices = np.array(df['Price']).reshape(-1, 1)

In [39]:
#Set min-max scaler to same as model
model_data = pd.read_csv('model_data.csv')
model_prices = np.array(model_data['price'])

# Load the same MinMaxScaler used during training
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = RobustScaler()
scaler.fit(model_prices.reshape(-1, 1))  # Fit on the current live prices

MinMaxScaler()

In [40]:
#read test data
df = pd.read_csv('./recent_eth_prices.csv')
df.head()

,Unnamed: 0,trade_id,product_id,price,size,time,side,exchange,bid,ask
0,0,610378394,ETH-USD,2693.31,0.002662,2025-02-15 23:04:16.929545+00:00,SELL,NaN,NaN,NaN
1,1,610378498,ETH-USD,2693.07,0.003294,2025-02-15 23:04:31.279857+00:00,SELL,NaN,NaN,NaN
2,2,610378533,ETH-USD,2693.89,0.003243,2025-02-15 23:04:46.953075+00:00,SELL,NaN,NaN,NaN
3,3,610378584,ETH-USD,2693.66,0.002293,2025-02-15 23:05:01.536506+00:00,SELL,NaN,NaN,NaN
4,4,610378657,ETH-USD,2693.98,0.000537,2025-02-15 23:05:16.783522+00:00,SELL,NaN,NaN,NaN


In [41]:
# Scale prices to match training format
prices_original = df['price'].values.reshape(-1, 1)
df['transformed_price'] = scaler.transform(prices_original)
df['sma_5'] = df['transformed_price'].rolling(window=5).mean()
df['momentum'] = df['transformed_price'] - df['transformed_price'].shift(5)
df['price_scaled'] = scaler.fit_transform(df[['price']])
df['pct_change'] = df['price'].pct_change() * 100  # Convert to percentage
df.dropna(subset=['sma_5', 'momentum'], inplace=True)
df.head()

prices = df['price_scaled'].values
sma_5 = df['sma_5'].values
momentum = df['momentum'].values

In [42]:
# Prepare the data for LSTM (same sequence length used in training)
X_test = []
last_prices = []

# Create sequences of data for LSTM
sequence_length = 10  # Sequence of 10 observations
forecast_steps = 5  # Predict 5 steps into the future

for i in range(0, len((prices) - sequence_length - forecast_steps), sequence_length):
    try:
        row = []
        row.append(np.array(prices[i:i + sequence_length]).reshape(sequence_length, 1))

        X_test.append(prices[i:i + sequence_length])
        last_prices.append(prices_original[i + sequence_length])
    except ValueError:
        break

X_test = np.array(X_test)

In [43]:
# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions to get actual price values
predicted_prices = scaler.inverse_transform(predictions)

442/442 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [44]:
starting_value = 10000
wallet = PaperWallet(starting_usd=starting_value)

In [45]:
len(prices)

141248

In [46]:
#run through prices, make predictions, and modify paper wallet as such
sequence_length = 10  # Sequence of 10 observations
forecast_steps = 5  # Predict 5 steps into the future

buy_transactions = 0
sell_transactions = 0

for i in range(len(predicted_prices)):
        #build feature set
        predicted_price_transformed = predicted_prices[i]
        last_price = last_prices[i]
        buy_eth = predicted_price_transformed > last_price

        #amount of wallet to trade on each transaction
        fraction = 0.2

        if buy_eth:
            usd_total = wallet.usd_balance
            trade_amount = usd_total * fraction

            wallet.buy_eth(trade_amount, last_price)
            buy_transactions += 1
        else:
            eth_total = wallet.eth_balance
            trade_amount = eth_total * fraction

            wallet.sell_eth(trade_amount, last_price)
            sell_transactions += 1

print(f"Buy Transactions: {buy_transactions} Sell Transactions: {sell_transactions}")
wallet.print_total_value(last_price)

Buy Transactions: 9041 Sell Transactions: 5083
Total Value: $[27683.35201424]
Percent Difference: [176.83352014]
